## JSON parser


此输出解析器允许用户指定任意 JSON 架构并查询 LLMs 以获得符合该架构的输出。

请记住，大型语言模型是有漏洞的抽象！您必须使用具有足够容量的 LLM 来生成格式正确的 JSON。在OpenAI家族中，达芬奇可以可靠地完成任务，但居里的能力已经急剧下降。

您可以选择使用 Pydantic 来声明您的数据模型。

In [1]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3)

/Users/libing/miniconda3/envs/kk_agent/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# 自定义一个数据结构
class Book(BaseModel):
    title: str = Field(description="书名")
    author: str = Field(description="作者")
    publication_year: int = Field(description="书的简介")

quary = "请告诉学习中国历史的经典书籍"

parser = JsonOutputParser(pydantic_object=Book)


In [3]:
format_instructions = parser.get_format_instructions()

format_instructions = """输出应格式化为符合以下 JSON 结构的 JSON 实例。
JSON结构
```
{
'title': '书的标题',
'author': '作者',
'description': '书的简介'
}
```
"""

prompt = PromptTemplate(
    template =  "{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions": format_instructions}
)

chain = prompt | chat | parser

chain.invoke({"query": quary})


{'title': '《中国通史》',
 'author': '吕思勉',
 'description': '这本书是中国现代著名的历史学家吕思勉先生所著的一部全面介绍中国历史的著作。它涵盖了从远古时期到民国初年的中国历史，内容详实、论述严谨，是了解和学习中国历史的重要参考书之一。'}

In [4]:
# 流式输出

for chunk in chain.stream({"query": quary}):
    print(chunk, end="", flush=True)

{}{'title': ''}{'title': '《'}{'title': '《中国'}{'title': '《中国通'}{'title': '《中国通史'}{'title': '《中国通史》'}{'title': '《中国通史》', 'author': ''}{'title': '《中国通史》', 'author': '吕'}{'title': '《中国通史》', 'author': '吕思'}{'title': '《中国通史》', 'author': '吕思勉'}{'title': '《中国通史》', 'author': '吕思勉', 'description': ''}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国现代'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国现代著名'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国现代著名的历史'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国现代著名的历史学家'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国现代著名的历史学家吕'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国现代著名的历史学家吕思'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国现代著名的历史学家吕思勉'}{'title': '《中国通史》', 'author': '吕思勉', 'description': '这本书是中国现代著名的历史学家吕思勉先生'}{'title': '《中国通史》', 'author': '吕

## 修复输出解析器

此输出解析器包装另一个输出解析器，如果第一个输出解析器失败，它会调用另一个 LLM 来修复任何错误。

但除了抛出错误之外，我们还可以做其他事情。具体来说，我们可以将格式错误的输出以及格式化的指令传递给模型并要求其修复。

对于这个例子，我们将使用上面的 Pydantic 输出解析器。如果我们传递一个不符合模式的结果，会发生以下情况：

In [5]:
from typing import List

class Actor(BaseModel):
    name: str = Field(description="演员的名字")
    film_names: List[str] = Field(description="演员出演的电影")

from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Actor)

actor_quary = "生成随机演员的电影作品表"

In [6]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']"
parser.parse(misformatted)

OutputParserException: Failed to parse Actor from completion {}. Got: 2 validation errors for Actor
name
  field required (type=value_error.missing)
film_names
  field required (type=value_error.missing)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [10]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, 
                                         llm=chat)

In [11]:
new_parser.parse(misformatted)

KeyError: "Input to PromptTemplate is missing variables {'instructions'}.  Expected: ['completion', 'error', 'instructions'] Received: ['completion', 'error']\nNote: if you intended {instructions} to be part of the string and not a variable, please escape it with double curly braces like: '{{instructions}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "